In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
tf.__version__

'1.15.2'

In [4]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [5]:
from tensorflow.keras.datasets import mnist
from tensorflow.contrib.layers import batch_norm, fully_connected
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.decomposition import PCA, IncrementalPCA
from sklearn.model_selection import train_test_split
import cv2

reset_graph()
'''Preparing==============================================='''
(X_data, y_label), (_, _) = mnist.load_data()
n_inputs = 28*28
n_hidden1 = 150
n_hidden2 = 100
n_hidden3 = 50
n_outputs = 10

X_data = np.array(X_data)
y_label = np.array(y_label)

X_data = X_data.reshape(len(X_data), 28*28).astype(np.float32)/255.0
y_label = y_label.astype(np.int32)
# X_test = X_test.reshape(len(X_test), 28*28).astype(np.float32)/255.0
# y_test = y_test[:n_test].astype(np.int32)

def convert_pca(X, n_batches=100, n_component=154):
  pca = PCA()
  inc_pca = IncrementalPCA(n_components=n_component)
  for X_batch in np.array_split(X, n_batches):
    print('.', end='')
    inc_pca.partial_fit(X_batch)
  return inc_pca.transform(X)

X_data = convert_pca(X=X_data)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size=0.33, random_state=42)

m_train, n_train = X_train.shape

n_epochs = 6
batch_size = 50
n_batches = int(np.ceil(m_train/batch_size))

def fetch_batch(epoch, batch_index):
    np.random.seed(epoch*n_batches + batch_index)
    indices = np.random.randint(m_train, size=batch_size)
    X_batch = X_train[indices]
    y_batch = y_train[indices]
    return X_train, y_train

'''placeholder==============================================='''
X = tf.placeholder(dtype=tf.float32, shape=(None, 154), name='X')
y = tf.placeholder(dtype=tf.int32, shape=(None), name='y')

is_training = tf.placeholder_with_default(False, shape=(), name='is_training')
bn_params = {
 'is_training': is_training,
 'decay': 0.99,
 'updates_collections': None
}

'''hidden layer=================================================================================================='''
hidden1 = fully_connected(X, n_hidden1, scope="hidden1", normalizer_fn=batch_norm, normalizer_params=bn_params)

hidden2 = fully_connected(hidden1, n_hidden2, scope="hidden2", normalizer_fn=batch_norm, normalizer_params=bn_params)

hidden3 = fully_connected(hidden2, n_hidden3, scope="hidden3", normalizer_fn=batch_norm, normalizer_params=bn_params)

logits = fully_connected(hidden3, n_outputs, activation_fn=None,scope="outputs", normalizer_fn=batch_norm, normalizer_params=bn_params)

'''Loss======================================='''
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name='loss')

'''optimizer==============================================================='''
def learning_schedule(t):
    return 1 / (100 * (t+1))

t = tf.placeholder(dtype=tf.float32, shape=(None))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_schedule(t))
training_op = optimizer.minimize(loss)

'''accuracy======================================================================================'''
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index)
            _accuracy, _loss, _ = sess.run([accuracy, loss, training_op], feed_dict={X: X_batch, y: y_batch, t: epoch})
        _test_accuracy = sess.run(accuracy, feed_dict={X: X_test, y: y_test})
        print("Epoch: {} \ttLoss: {}, \tTrain_Accuracy: {} \tTest_Accuracy: {}".format(epoch, _loss, _accuracy*100, _test_accuracy*100))
    save_path = saver.save(sess, './my_model.ckpt')

11493376/11490434 [==============================] - 0s 0us/step
Instructions for updating:
Please use `layer.__call__` method instead.
Epoch: 0 	tLoss: 1.2574575521284714e-05, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.07070589065552
Epoch: 1 	tLoss: 5.122397396917222e-06, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.05050587654114
Epoch: 2 	tLoss: 2.9776833798678126e-06, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.04545736312866
Epoch: 3 	tLoss: 1.958251004907652e-06, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.04040288925171
Epoch: 4 	tLoss: 1.3616039495900623e-06, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.04545736312866
Epoch: 5 	tLoss: 9.723722769194865e-07, 	Train_Accuracy: 100.0 	Test_Accuracy: 97.05555438995361


In [6]:
with tf.Session() as sess:
  saver.restore(sess, 'my_model.ckpt')
  output = logits.eval(feed_dict={X: X_test[2].reshape(1, 154)})

INFO:tensorflow:Restoring parameters from my_model.ckpt


In [7]:
print('True: {} \t\t Test: {}'.format(y_test[2], np.argmax(output)))

True: 8 		 Test: 8
